In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb

In [78]:
data=pd.read_csv("cleaned_train.csv")

In [31]:
data.shape

(1025272, 19)

In [32]:
#data=sepclass(data)

In [33]:
data.shape

(1553458, 19)

In [79]:
del data['Unnamed: 0']

In [80]:

y=data["cancel"]
X=data.drop(["cancel","zip.code"],axis=1)
dummyx=pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(dummyx, y, test_size=0.2, random_state=1)

In [81]:
d=pd.concat([y_train,X_train],axis=1)
d=sepclass(d)
y_train=d["cancel"]
X_train=d.drop(["cancel"],axis=1)

In [82]:
data.head(2)["cancel"]

0    0
1    0
Name: cancel, dtype: int64

In [83]:
X_test.head(3)

,year,ni.age,len.at.res,premium,ni.marital.status,n.adults,n.children,tenure,claim.ind,house.color_blue,...,sales.channel_Online,sales.channel_Phone,ni.gender_F,ni.gender_M,state_AZ,state_CO,state_IA,state_PA,state_VA,state_WA
175475,2015,53.0,16.0,883.480110,1.0,1.0,5.0,11.0,0.0,0,...,0,0,1,0,0,1,0,0,0,0
59936,2013,58.0,17.0,855.929822,0.0,3.0,0.0,10.0,1.0,0,...,0,0,1,0,0,0,0,1,0,0
689171,2016,28.0,15.0,1004.368385,1.0,2.0,0.0,7.0,1.0,0,...,0,0,1,0,0,0,0,1,0,0


In [58]:
#y_test=y_test.append(pd.DataFrame(data = X_test), ignore_index=True)

/Users/westspringamerica/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [84]:
sepclass(data).shape

(1553458, 18)

In [ ]:
#tree model

In [85]:
from sklearn import tree
blf = tree.DecisionTreeClassifier()
modelx=blf.fit(X_train,y_train)
y_predict=modelx.predict(X_test)

In [86]:
fpr, tpr, thresholds = metrics.roc_curve( y_predict,y_test, pos_label=1)
metrics.auc(fpr, tpr)

0.5620748491458918

In [ ]:
#0.5616306155395154  0.866459758411376

In [ ]:
#xgboost model

In [87]:
data_train = xgb.DMatrix(X_train, y_train)  # 使用XGBoost的原生版本需要对数据进行转化
data_test = xgb.DMatrix(X_test, y_test)

In [88]:
param = {'max_depth': 5, 'eta': 1, 'objective': 'binary:logistic'}
watchlist = [(data_test, 'test'), (data_train, 'train')]
n_round = 3
booster = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist)

[0]	test-error:0.40869	train-error:0.346439
[1]	test-error:0.415966	train-error:0.338564
[2]	test-error:0.388769	train-error:0.331063


In [89]:
# error rate
pred = booster.predict(data_test)
y = data_test.get_label()

In [90]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.7251090391772068

In [ ]:
#0.7223003128577246 0.7251090391772068

In [19]:
from sklearn.utils import resample
def sepclass(df):
    # Separate majority and minority classes
    df_majority = df[df.cancel==0]
    df_minority = df[df.cancel==1]

    # Upsample minority class
    df_minority_upsampled = resample(df_minority, 
                                     replace=True,     # sample with replacement
                                     n_samples=776729,    # to match majority class
                                     random_state=123) # reproducible results

    # Combine majority class with upsampled minority class
    df_upsampled = pd.concat([df_majority, df_minority_upsampled])

    # Display new class counts
    df_upsampled.cancel.value_counts()
    # 1    776729
    # 0    776729
    # Name: balance, dtype: int64

    #print(df_upsampled.head())
    return(df_upsampled)